# Haris角点和SIFT特征描述符

## 这个示例展示一个在AOP上运行的，特征点检测和匹配算法，RGB上的数据用于验证

调用接口：
- tianmoucv.proc.features.(HarrisCorner,sift,hog)
- tianmoucv.proc.tracking.(feature_matching,mini_l2_cost_matching,align_images)

In [ ]:
%load_ext autoreload

## 必要的包

In [ ]:
%autoreload
import sys,os
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tianmoucv.isp import lyncam_raw_comp,demosaicing_npy,SD2XY
from tianmoucv.proc.features import HarrisCorner,sift,hog
from tianmoucv.proc.tracking import feature_matching,mini_l2_cost_matching,align_images
import cv2
from tianmoucv.data import TianmoucDataReader



## 准备数据

In [ ]:
train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]

val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []
print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
        
all_data = valdata + traindata
key_list = ['underbridge_hdr_4']

# 特征点检测和匹配，利用相隔150帧的SD做匹配

In [ ]:
%autoreload
import time
from tianmoucv.proc.reconstruct import laplacian_blending
from IPython.display import clear_output

key= 'outdoor_bridge_3'
dataset = TianmoucDataReader(all_data,MAXLEN=500*1,matchkey=key,speedUpRate=1)

imlist = []
for sampleid in range(len(dataset)):
    if sampleid<95:
        continue
    if sampleid>99:
        break
    else:
        ###############################
        clear_output()
        sample = dataset[sampleid]
        F1 = sample['F0']
        tsdiff = sample['rawDiff']/255.0
        threshed_tsdiff = tsdiff[:,0,...].permute(1,2,0)
        SD = threshed_tsdiff[...,1:]
        TD = threshed_tsdiff[...,0]
        Ix1,Iy1= SD2XY(SD)
        gray = laplacian_blending(-Ix1,-Iy1,iteration=20)
        gray_laplac1 = F.interpolate(torch.Tensor(gray).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)

        # 选择5帧后的数据做匹配验证
        sample = dataset[sampleid+5]
        F2 = sample['F0']
        tsdiff = sample['rawDiff']/255.0
        threshed_tsdiff = tsdiff[:,0,...].permute(1,2,0)
        SD = threshed_tsdiff[...,1:]
        TD = threshed_tsdiff[...,0]
        Ix2,Iy2= SD2XY(SD)
        gray = laplacian_blending(-Ix2,-Iy2,iteration=20)
        gray_laplac2 = F.interpolate(torch.Tensor(gray).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)

        gray_laplac1 = (gray_laplac1 - torch.min(gray_laplac1)) / (torch.max(gray_laplac1) - torch.min(gray_laplac1) + 1e-3) 
        gray_laplac2 = (gray_laplac2 - torch.min(gray_laplac2)) / (torch.max(gray_laplac2) - torch.min(gray_laplac2) + 1e-3) 
        
        F1_g = torch.stack([gray_laplac1]*3,dim=-1)
        F2_g = torch.stack([gray_laplac2]*3,dim=-1)

        ## 计算harris角点
        featureList1 = []
        kp1=[]
        featureList2 = []
        kp2=[]
        k = 0.01
        th = 0.01
        nmsSize= 15
        startT = time.time()
        idmap,R = HarrisCorner(Ix1,Iy1,k=k,th=th,nmsSize=nmsSize)
        endT = time.time()
        idmap2,R = HarrisCorner(Ix2,Iy2,k=k,th=th,nmsSize=nmsSize)
        print('corner detect cost:',endT-startT,'s')

        canvas_rgb = np.zeros([320,1280,3])
        canvas_rgb[:,:640,:] = F1
        canvas_rgb[:,640:,:] = F2
        
        canvas = np.zeros([320,1280,3])
        canvas[:,:640,:] = F1_g.numpy()
        canvas[:,640:,:] = F2_g.numpy()
        
        #(step1)第一张图的feature list
        for i in range(idmap.shape[0]):
            for j in range(idmap.shape[1]):
                if idmap[i,j]>0:
                    cv2.circle(canvas,(j*2,i*2),4,(0,0,255))
                    kp1.append([i,j])
        #(step2)第二张图的feature list
        for i in range(idmap2.shape[0]):
            for j in range(idmap2.shape[1]):
                if idmap2[i,j]>0:
                    cv2.circle(canvas,(j*2+640,i*2),4,(255,0,0))
                    kp2.append([i,j])

        #(step3)计算两张图对应fp list的特征描述子
        startT = time.time()
        kp1,featureList1 = sift(Ix1,Iy1,kp1)
        endT = time.time()
        print('feature extract cost:',endT-startT,'s')
        kp2,featureList2 = sift(Ix2,Iy2,kp2) 
        print('good kp1/2:',len(kp1),len(kp2))
        #print('>>>>>KP1:',kp1)
        #print('>>>>>KP2:',kp2)
        #mapping to rgb coordinate
        kp1 = [ (p[0]*2,p[1]*2) for p in kp1 ]
        kp2 = [ (p[0]*2,p[1]*2) for p in kp2 ]
        fl1 = torch.stack(featureList1,dim=0).view(len(kp1),-1)
        fl2 = torch.stack(featureList2,dim=0).view(len(kp2),-1)

        #(step4)匹配特征
        startT = time.time()
        matches = feature_matching(fl1, fl2, ratio=0.75)
        endT = time.time()
        print('matched kp:',len(matches),'cost time:',endT-startT,'s')
        canvas_wp, H = align_images(F1.copy(),kp1, kp2, matches, canvas) 

        
        plt.figure(figsize=(9,3))
        plt.imshow(canvas)
        plt.show()
        imlist.append(canvas)
        
        plt.figure(figsize=(9,3))
        plt.imshow(canvas_rgb)
        plt.show()

        plt.figure(figsize=(9,3))
        plt.subplot(1,2,1)  
        plt.imshow(canvas_wp)
        plt.subplot(1,2,2)  
        plt.imshow((F2+canvas_wp)/2)
        plt.show()


In [ ]:
def images_to_video(frame_list,name,Val_size=(512,256),Flip=False):
    fps = 30     
    size = (Val_size[0], Val_size[1]*2) # 需要转为视频的图片的尺寸
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for frame in frame_list:
        frame = (frame-np.min(frame))/(np.max(frame)-np.min(frame)) * 255
        frame2 = frame.astype(np.uint8)
        out.write(frame2)
    out.release()
    
images_to_video(imlist,'./'+key+'.mp4',Val_size=(640,320),Flip=False)